# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudness,Wind speed,Country,Date
0,0,east london,-33.0153,27.9116,63.77,53,2,5.44,ZA,1623271182
1,1,upernavik,72.7868,-56.1549,36.28,87,100,2.51,GL,1623271191
2,2,mutata,7.2441,-76.4356,77.92,86,99,3.04,CO,1623271377
3,3,cetraro,39.5166,15.9416,67.24,88,85,3.04,IT,1623271377
4,4,kaitangata,-46.2817,169.8464,55.92,75,100,7.76,NZ,1623271208
...,...,...,...,...,...,...,...,...,...,...
556,556,manutuke,-38.6833,177.9167,48.97,89,20,5.70,NZ,1623271483
557,557,beloha,-25.1667,45.0500,65.66,77,13,13.82,MG,1623271325
558,558,taree,-31.9000,152.4667,40.57,100,82,0.00,AU,1623271484
559,559,kendallville,41.4414,-85.2650,86.02,55,40,3.44,US,1623271329


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)
#print(api_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
#print(locations)

# Store Humidity in humidity
humidity = weather_data["Humidity"]
#print(humidity)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

max_intensity = np.max(humidity)
#print(max_intensity)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind speed"] <= 10) & (weather_data["Cloudness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudness,Wind speed,Country,Date
53,53,chardara,41.2547,67.9692,76.95,26,0,5.79,KZ,1623271383
112,112,quartucciu,39.2519,9.1794,73.27,64,0,7.52,IT,1623271291
159,159,benguela,-12.5763,13.4055,74.50,64,0,1.66,AO,1623271402
468,468,santa lucia,27.9117,-15.5407,71.94,43,0,5.95,ES,1623271466
470,470,catamarca,-28.4696,-65.7852,71.78,62,0,1.99,AR,1623271289


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,chardara,KZ,41.2547,67.9692,
112,quartucciu,IT,39.2519,9.1794,
159,benguela,AO,-12.5763,13.4055,
468,santa lucia,ES,27.9117,-15.5407,
470,catamarca,AR,-28.4696,-65.7852,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    #locations = weather_data[["Lat", "Lng"]]
    #print(params)
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #print(response)
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 53: chardara.
Closest hotel in chardara is Жанаргул.
------------
Retrieving Results for Index 112: quartucciu.
Closest hotel in quartucciu is Palazzo Doglio.
------------
Retrieving Results for Index 159: benguela.
Closest hotel in benguela is Aparthotel Mil Cidades.
------------
Retrieving Results for Index 468: santa lucia.
Closest hotel in santa lucia is Las Tirajanas.
------------
Retrieving Results for Index 470: catamarca.
Closest hotel in catamarca is Amerian Catamarca Park Hotel.
------------
-------End of Search-------


In [11]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,chardara,KZ,41.2547,67.9692,Жанаргул
112,quartucciu,IT,39.2519,9.1794,Palazzo Doglio
159,benguela,AO,-12.5763,13.4055,Aparthotel Mil Cidades
468,santa lucia,ES,27.9117,-15.5407,Las Tirajanas
470,catamarca,AR,-28.4696,-65.7852,Amerian Catamarca Park Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map (fig)
fig

Figure(layout=FigureLayout(height='420px'))